In [1]:
import pyspark.sql
from pyspark.sql import functions as F
import pandas as pd

# 1. Read the case, department, and source data into their own spark dataframes.

In [2]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/22 10:15:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
pd_cases = pd.read_csv('311.csv')
pd_dept = pd.read_csv('dept.csv')

In [4]:
cases = spark.createDataFrame(pd_cases)

In [5]:
dept = spark.createDataFrame(pd_dept)

In [6]:
cases.describe().show(vertical=True)

23/05/22 10:16:04 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
23/05/22 10:16:06 WARN TaskSetManager: Stage 0 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


-RECORD 0------------------------------------
 summary              | count                
 case_id              | 100000               
 case_opened_date     | 100000               
 case_closed_date     | 100000               
 SLA_due_date         | 100000               
 case_late            | 100000               
 num_days_late        | 100000               
 case_closed          | 100000               
 dept_division        | 100000               
 service_request_type | 100000               
 SLA_days             | 100000               
 case_status          | 100000               
 source_id            | 100000               
 request_address      | 100000               
 council_district     | 100000               
-RECORD 1------------------------------------
 summary              | mean                 
 case_id              | 1.01422656004933E9   
 case_opened_date     | null                 
 case_closed_date     | NaN                  
 SLA_due_date         | NaN       

In [7]:
dept.describe().show(vertical=True)

-RECORD 0--------------------------------------
 summary                | count                
 dept_division          | 39                   
 dept_name              | 39                   
 standardized_dept_name | 39                   
 dept_subject_to_SLA    | 39                   
-RECORD 1--------------------------------------
 summary                | mean                 
 dept_division          | null                 
 dept_name              | NaN                  
 standardized_dept_name | null                 
 dept_subject_to_SLA    | null                 
-RECORD 2--------------------------------------
 summary                | stddev               
 dept_division          | null                 
 dept_name              | null                 
 standardized_dept_name | null                 
 dept_subject_to_SLA    | null                 
-RECORD 3--------------------------------------
 summary                | min                  
 dept_division          | 311 Call Cente

## 2. Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json

Im going to do this with dept instead

In [8]:
dept.write.csv('data/dept', mode='overwrite', header=True)

In [9]:
dept.write.json('data/dept', mode='overwrite')

### Inspect your folder structure. What do you notice?
The files are split up into smaller files

## 3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [10]:
cases.dtypes

[('case_id', 'bigint'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('SLA_due_date', 'string'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'string'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'bigint')]

In [11]:
cases.select('case_opened_date').show(2)

+----------------+
|case_opened_date|
+----------------+
|     1/1/18 0:42|
|     1/1/18 0:46|
+----------------+
only showing top 2 rows



23/05/22 10:16:40 WARN TaskSetManager: Stage 8 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


In [12]:
fmt = 'M/d/yy H:m'

In [13]:
cases = cases.withColumn(
    'case_opened_date',
    F.to_timestamp('case_opened_date', fmt)
    ).\
    withColumn(
    'case_closed_date',
    F.to_timestamp('case_closed_date', fmt)
    ).\
    withColumn(
    'SLA_due_date',
    F.to_timestamp('SLA_due_date', fmt)
    )

In [14]:
cases.select('case_opened_date').show(2)

23/05/22 10:16:40 WARN TaskSetManager: Stage 9 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


+-------------------+
|   case_opened_date|
+-------------------+
|2018-01-01 00:42:00|
|2018-01-01 00:46:00|
+-------------------+
only showing top 2 rows



In [15]:
cases.select('SLA_due_date').show(3)

+-------------------+
|       SLA_due_date|
+-------------------+
|2020-09-26 00:42:00|
|2018-01-05 08:30:00|
|2018-01-05 08:30:00|
+-------------------+
only showing top 3 rows



23/05/22 10:16:41 WARN TaskSetManager: Stage 10 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


In [16]:
cases.dtypes

[('case_id', 'bigint'),
 ('case_opened_date', 'timestamp'),
 ('case_closed_date', 'timestamp'),
 ('SLA_due_date', 'timestamp'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'string'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'bigint')]

In [17]:
cases.select('case_late', 'case_closed', 'case_status').show(3)

+---------+-----------+-----------+
|case_late|case_closed|case_status|
+---------+-----------+-----------+
|       NO|        YES|     Closed|
|       NO|        YES|     Closed|
|       NO|        YES|     Closed|
+---------+-----------+-----------+
only showing top 3 rows



23/05/22 10:16:41 WARN TaskSetManager: Stage 11 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


In [18]:
cases.select('case_status').distinct().show()

23/05/22 10:16:41 WARN TaskSetManager: Stage 12 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


+-----------+
|case_status|
+-----------+
|       Open|
|     Closed|
+-----------+



case_status is redundant with case_closed

In [19]:
cases = cases.withColumn(
    'case_closed', 
    F.expr('case_closed') == 'YES').\
    withColumn(
    'case_late', 
    F.expr('case_late') == 'YES')

In [20]:
cases = cases.drop('case_status')

In [21]:
cases.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 SLA_due_date         | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616         
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
only showing top 1 row



23/05/22 10:16:43 WARN TaskSetManager: Stage 15 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


In [22]:
cases = cases.withColumn(
    'request_address',
    F.trim(F.lower('request_address'))
)

In [23]:
cases.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 SLA_due_date         | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616         
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 5                    
only showing top 1 row



23/05/22 10:16:43 WARN TaskSetManager: Stage 16 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


# part 2
## 1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [26]:
cases.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 SLA_due_date         | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616         
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 5                    
only showing top 1 row



23/05/22 10:24:41 WARN TaskSetManager: Stage 18 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


In [38]:
# How old is the latest (in terms of days past SLA) currently open issue?
cases.filter('case_closed == False').\
    select(
        F.datediff(
            F.current_timestamp(),
            F.max('case_opened_date')).\
    alias('latest_open_days_open')
    ).show()

23/05/22 10:36:41 WARN TaskSetManager: Stage 41 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


+---------------------+
|latest_open_days_open|
+---------------------+
|                 1896|
+---------------------+



In [41]:
# How long has the oldest (in terms of days since opened) 
# currently opened issue been open?
cases.filter('case_closed == False').\
    select(
        F.datediff(
            F.current_timestamp(),
            F.min('case_opened_date')).\
    alias('oldest_open_days_open')
    ).show()

23/05/22 10:40:25 WARN TaskSetManager: Stage 50 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


+---------------------+
|oldest_open_days_open|
+---------------------+
|                 1966|
+---------------------+



## 2. How many Stray Animal cases are there?

In [55]:
cases.filter(F.col('dept_division') == 'Field Operations').\
    groupby('service_request_type').count().show()

23/05/22 10:47:42 WARN TaskSetManager: Stage 75 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|Animal Permits Re...|  380|
|Injured Animal(Cr...| 1021|
|     Officer Standby|  384|
|Animal Bite(Non-C...|  544|
|        Stray Animal| 3497|
|Trapped/Confined ...| 1299|
|City Council Anim...|   46|
|Aggressive Animal...| 3541|
|Animal Bite(Criti...|   82|
|Public Nuisance(O...| 1531|
|Aggressive Animal...|  746|
|Animal Cruelty(Cr...|  479|
|      Animal Neglect| 1928|
+--------------------+-----+



In [49]:
cases.filter(F.col('service_request_type') == 'Stray Animal').count()

23/05/22 10:44:16 WARN TaskSetManager: Stage 60 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


3497

#### 3497

## 3. How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [53]:
cases.filter(F.col('dept_division') == 'Field Operations').\
    filter(F.col('service_request_type') != 'Officer Standby').\
    count()

23/05/22 10:46:38 WARN TaskSetManager: Stage 72 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


15094

#### 15,094

## 4. Convert the council_district column to a string column.

In [61]:
cases = cases.withColumn(
    'council_district',
    F.col('council_district').cast('string')
)
cases.dtypes

[('case_id', 'bigint'),
 ('case_opened_date', 'timestamp'),
 ('case_closed_date', 'timestamp'),
 ('SLA_due_date', 'timestamp'),
 ('case_late', 'boolean'),
 ('num_days_late', 'double'),
 ('case_closed', 'boolean'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'string')]

## 5. Extract the year from the case_closed_date column.

In [65]:
cases = cases.withColumn(
    'case_closed_year',
    F.year(F.col('case_closed_date'))
)
cases.show(1,vertical=True)

23/05/22 10:54:52 WARN TaskSetManager: Stage 83 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 SLA_due_date         | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616         
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 5                    
 case_closed_year     | 2018                 
only showing top 1 row



## 6. Convert num_days_late from days to hours in new columns num_hours_late.

In [75]:
cases = cases.withColumn(
    'num_hours_late',
    F.round((F.col('num_days_late') * 24), 2)
)
cases.show(1, vertical=True)

23/05/22 11:05:40 WARN TaskSetManager: Stage 89 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 SLA_due_date         | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616         
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 5                    
 case_closed_year     | 2018                 
 num_hours_late       | -23964.21            
only showing top 1 row



## 7. Join the case data with the source and department data.

#### Don't have access to the source data...

## 8. Are there any cases that do not have a request source?

In [91]:
cases.filter(F.col('source_id').isNull()).count()

23/05/22 11:13:01 WARN TaskSetManager: Stage 95 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


0

#### None with null values

## 9. What are the top 10 service request types in terms of number of requests?

In [115]:
cases.groupby('service_request_type').\
    agg(F.count('service_request_type').\
        alias('counts')).\
    sort(F.col('counts').desc()).\
    show(10)

23/05/22 11:29:39 WARN TaskSetManager: Stage 119 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+------+
|service_request_type|counts|
+--------------------+------+
|           No Pickup|  8932|
|Overgrown Yard/Trash|  6853|
|Front Or Side Yar...|  5338|
|        Bandit Signs|  4781|
|Aggressive Animal...|  3541|
|        Stray Animal|  3497|
|        Damaged Cart|  3465|
|Cart Exchange Req...|  3282|
|Right Of Way/Side...|  3121|
|Junk Vehicle On P...|  2897|
+--------------------+------+
only showing top 10 rows



## 10. What are the top 10 service request types in terms of average days late?

In [123]:
cases.filter(F.col('service_request_type') != 'Request for Research/Information').\
    groupby('service_request_type').\
    agg(F.round(
            F.avg('num_days_late')\
        , 2).\
        alias('avg_days_late')
        ).\
    sort(F.col('avg_days_late').desc()).\
    show(10, truncate=False)

23/05/22 11:33:52 WARN TaskSetManager: Stage 134 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


+----------------------------------------+-------------+
|service_request_type                    |avg_days_late|
+----------------------------------------+-------------+
|Zoning: Junk Yards                      |178.32       |
|Emergency: Main and Accessory Structures|173.51       |
|Status: On-Going CIMS Projects          |159.86       |
|Zoning: Recycle Yard                    |152.18       |
|Complaint                               |136.78       |
|Markings Installation SMO (NEW)         |111.38       |
|Vendors                                 |71.22        |
|Donation Container Enforcement          |61.65        |
|No Address Posted                       |61.14        |
|Curbstoning                             |54.43        |
+----------------------------------------+-------------+
only showing top 10 rows



## 11. Does number of days late depend on department?

In [125]:
cases.groupby('dept_division').\
    agg(F.round(
            F.avg('num_days_late')\
        , 2).\
        alias('avg_days_late')
        ).\
    sort(F.col('avg_days_late').desc()).\
    show(truncate=False)

23/05/22 11:35:36 WARN TaskSetManager: Stage 140 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


+-----------------------------+-------------+
|dept_division                |avg_days_late|
+-----------------------------+-------------+
|District 2                   |NaN          |
|District 1                   |NaN          |
|Director's Office Horizontal |159.86       |
|Code Enforcement (Internal)  |152.18       |
|311 Call Center              |111.96       |
|Engineering Division         |30.16        |
|Trades                       |10.28        |
|Solid Waste                  |2.61         |
|Clean and Green Natural Areas|2.37         |
|Miscellaneous                |-1.72        |
|Tree Crew                    |-1.73        |
|Facility License             |-1.77        |
|Waste Collection             |-1.84        |
|Vector                       |-2.17        |
|Shops                        |-2.97        |
|Clean and Green              |-3.23        |
|Brush                        |-3.87        |
|Food Establishments          |-7.16        |
|Shops (Internal)             |-7.

#### Yes, there are 3 departments that average over 100 hours late, 1 department with avg 30 hours, 1 with 10 hours avg, and the rest below 5 avg hours late.

## 12. How do number of days late depend on department and request type?

In [126]:
cases.groupby('dept_division', 'service_request_type').\
    agg(F.round(
            F.avg('num_days_late')\
        , 2).\
        alias('avg_days_late')
        ).\
    sort(F.col('avg_days_late').desc()).\
    show(truncate=False)

23/05/22 11:38:07 WARN TaskSetManager: Stage 143 contains a task of very large size (1346 KiB). The maximum recommended task size is 1000 KiB.


+----------------------------+----------------------------------------+-------------+
|dept_division               |service_request_type                    |avg_days_late|
+----------------------------+----------------------------------------+-------------+
|District 2                  |Request for Research/Information        |NaN          |
|District 1                  |Request for Research/Information        |NaN          |
|Code Enforcement            |Zoning: Junk Yards                      |178.32       |
|Dangerous Premise           |Emergency: Main and Accessory Structures|173.51       |
|Director's Office Horizontal|Status: On-Going CIMS Projects          |159.86       |
|Code Enforcement (Internal) |Zoning: Recycle Yard                    |152.18       |
|311 Call Center             |Complaint                               |136.78       |
|Signs and Markings          |Markings Installation SMO (NEW)         |111.38       |
|Code Enforcement            |Vendors                 

#### When broken down, there are service types for 6 department that average over 100 hours late, and many others with an average of over 30 hours late